In [1]:
from wandb.sdk import wandb_config

import wandb
import torch
from model.train import train
from utils.configs import *
from data.CommentaryDataloader import get_commentary_dataloader
from data.CommentaryDataset import CommentaryDataset
from model.predict import Predictor
import wandb

In [2]:
def get_configs_from_wandb_config(wandb_config: WandbConfig, is_real_wandb: bool):
    if wandb_config['model_name'] not in ['model', 'model_residual_encoder']:
        raise ValueError('Model should be model or model_residual_encoder')
    
    shared_config: SharedConfig = {
        'context_length': wandb_config['context_length'],
        'sentencepiece_path': f"./artifacts/sp{wandb_config['sp_vocab']}.model",
        'bos_id': 0, # will be initialized later
        'eos_id': 1, # will be initialized later
        'pad_id': 2, # will be initialized later
        'vocab_size': 0 # will be initialized later
    }
    
    train_data_config: DataConfig = {
        'batch_size': wandb_config['batch_size'],
        'split': 'train',
        'data_path': './processed_data',
        'past_boards': wandb_config['past_boards'],
        'stride_big_sequences': wandb_config['stride_big_sequences'],
        'in_memory': False,
        'dl_shuffle': False,
        'dl_samples': wandb_config['samples_per_train_epoch'],
        'dl_num_workers': 2,
    }
    
    valid_data_config: DataConfig = {
        'batch_size': wandb_config['batch_size'],
        'split': 'valid',
        'data_path': './processed_data',
        'past_boards': wandb_config['past_boards'],
        'stride_big_sequences': wandb_config['stride_big_sequences'],
        'in_memory': True,
        'dl_shuffle': True,
        'dl_samples': None,
        'dl_num_workers': 1,
    }
    
    test_data_config: DataConfig = {
        'batch_size': wandb_config['batch_size'],
        'split': 'valid',
        'data_path': './processed_data',
        'past_boards': wandb_config['past_boards'],
        'stride_big_sequences': wandb_config['stride_big_sequences'],
        'in_memory': False,
        'dl_shuffle': False,
        'dl_samples': None,
        'dl_num_workers': 0,
    }
    
    model_config: ModelConfig = {
        'name': Models.MODEL_RESIDUAL_ENCODER,
        'board_embedding_size': wandb_config['board_embedding_size'],
        'text_embedding_size': wandb_config['text_embedding_size'],
        'conv_modules_count': wandb_config['conv_modules_count'],
        'transformer_blocks': wandb_config['transformer_blocks'],
        'board_intermediary_channels': wandb_config['board_intermediary_channels'],
        'board_in_channels': CommentaryDataset.get_board_channels(train_data_config),
        'board_height': 8,
        'board_width': 8,
        'board_depth': wandb_config['board_embedding_size'],
        'ff_inner_channels': wandb_config['ff_inner_channels'],
        'num_heads': wandb_config['num_heads']
    }
    
    if wandb_config['optimizer'] not in ['adam', 'sgd']:
        raise ValueError('Optimizer value invalid')
    
    train_config: TrainConfig = {
        'lr': wandb_config['lr'],
        'with_wandb': is_real_wandb,
        'num_epochs': wandb_config['num_epochs'],
        'predict_sentences': wandb_config['predict_sentences'],
        'optimizer': Optimizers.ADAM if wandb_config['optimizer'] == 'adam' else Optimizers.SGD
    }
    
    return {
        'shared_config': shared_config,
        'train_config': train_config,
        'model_config': model_config,
        'test_data_config': test_data_config,
        'train_data_config': train_data_config,
        'valid_data_config': valid_data_config
    }     

In [3]:
# local_wandb_config: WandbConfig = {
#     'text_embedding_size': 64,
#     'conv_modules_count': 6,
#     'transformer_blocks': 6,
#     'board_intermediary_channels': 64,
#     'board_embedding_size': 64,
#     'ff_inner_channels': 64,
#     'num_heads': 4,
#     'lr': 0.01,
#     'optimizer': 'sgd',
#     'num_epochs': 200,
#     'context_length': 256,
#     'sp_vocab': 800,
#     'batch_size': 64,
#     'past_boards': 1,
#     'stride_big_sequences': 64,
#     'samples_per_train_epoch': 100000,
#     'predict_sentences': 4
# }

sweep_config = {
    "method": "random",
    "metric": {"goal": "minimize", "name": "val_loss"},
    "parameters": {
        'model_name': {"values": ['model_residual_encoder', 'model'] },
        'text_embedding_size': {"distribution": "q_log_uniform_values", "min": 64, "max": 256, "q": 64},
        'conv_modules_count': {"values": [1, 2, 3, 4]},
        'transformer_blocks': {"values": [1, 2, 3, 4]},
        'board_intermediary_channels': {"distribution": "q_log_uniform_values", "min": 64, "max": 512, "q": 64},
        'board_embedding_size': {"distribution": "q_log_uniform_values", "min": 64, "max": 256, "q": 64},
        'ff_inner_channels': {"distribution": "q_log_uniform_values", "min": 64, "max": 512, "q": 64},
        'num_heads': {"distribution": "q_log_uniform_values", "min": 4, "max": 8, "q": 4},
        'lr': {"distribution": "uniform", "max": 0.4, "min": 0},
        'optimizer': {"values": ['sgd', 'adam']},
        'num_epochs': {"values": [2, 3, 4]},
        'context_length': {"distribution": "q_log_uniform_values", "min": 64, "max": 512, "q": 64},
        'sp_vocab': {"values": [700, 800, 900]},
        'batch_size': {"distribution": "q_log_uniform_values", "min": 64, "max": 256, "q": 64},
        'past_boards': {"values": [0, 1, 2]},
        'stride_big_sequences': {"values": [256, 512]},
        'samples_per_train_epoch': {"values": [10000, 100000]},
        'predict_sentences': {"values": [10]}
    }
}

In [4]:
def run(): 
    with wandb.init(project="thesis"):
        configs = get_configs_from_wandb_config(wandb.config, True)
        train(
            model_config=configs['model_config'],
            train_config=configs['train_config'],
            shared_config=configs['shared_config'],
            train_dl=get_commentary_dataloader(configs['train_data_config'], configs['shared_config'])[0],
            val_dl=get_commentary_dataloader(configs['valid_data_config'], configs['shared_config'])[0],
            test_ds=CommentaryDataset(configs['test_data_config'], configs['shared_config']),
            predictor=Predictor(configs['shared_config'])
        )

In [5]:
sweep_id = wandb.sweep(sweep_config, project = 'thesis')


Create sweep with ID: pj49gpx4
Sweep URL: https://wandb.ai/georgerapeanu/thesis/sweeps/pj49gpx4


In [6]:
wandb.agent(sweep_id, function=run)

wandb: Agent Starting Run: wwcspg7b with config:
wandb: 	batch_size: 64
wandb: 	board_embedding_size: 64
wandb: 	board_intermediary_channels: 64
wandb: 	context_length: 64
wandb: 	conv_modules_count: 4
wandb: 	ff_inner_channels: 128
wandb: 	lr: 0.10686326782814902
wandb: 	model_name: model
wandb: 	num_epochs: 2
wandb: 	num_heads: 8
wandb: 	optimizer: sgd
wandb: 	past_boards: 0
wandb: 	predict_sentences: 10
wandb: 	samples_per_train_epoch: 100000
wandb: 	sp_vocab: 900
wandb: 	stride_big_sequences: 256
wandb: 	text_embedding_size: 64
wandb: 	transformer_blocks: 4
wandb: Currently logged in as: georgerapeanu. Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Epoch 1/2: train_loss: 6.06767348364539, val_loss: 5.017977394434829
Past board None
Past evaluation 0
Current board r n b q k b n r
p p . p p p p p
. . . . . . . .
. . p . . . . .
. . . . P . . .
. . . P . . . .
P P P . . P P P
R N B Q K B N R
Current evaluation 16
Actual prediction This is not too bad, although not very common, as it is not very aggressive.
Predicted text Kfaven't maly exchange pawn sted, Bens my king move or thortpend.
Past board None
Past evaluation 0
Current board r n b q k b n r
p p p . . p p p
. . . . . . . .
. . . . p . . .
. . p P P . . .
. . . . . . . .
P P . . . P P P
R N B Q K B N R
Current evaluation -15
Actual prediction And she picks the most-played variation. If I now take on e5, she can exchange queens on d1 and then play Be6 protecting her pawn on
Predicted text Smits pother ma youurside.
Past board None
Past evaluation 0
Current board . . . . k . n r
p . r n . p p p
. . . . . q . .
. . b . p . . .
. . . . B . . .
. . . . . . . .
P P P P . P P P
R N B

wandb: Waiting for artifact model_val_loss to be committed...



wandb: Committed artifact georgerapeanu/thesis/model_val_loss:v43
wandb: Waiting for artifact model_train_loss to be committed...

wandb: Committed artifact georgerapeanu/thesis/model_train_loss:v43


train_loss,█▁
val_loss,█▁
train_loss,5.14112
val_loss,4.68882


...

wandb: Waiting for artifact model_val_loss to be committed...



wandb: Committed artifact georgerapeanu/thesis/model_val_loss:v50
wandb: Waiting for artifact model_train_loss to be committed...

wandb: Committed artifact georgerapeanu/thesis/model_train_loss:v50


train_loss,█▁
val_loss,▁█
train_loss,6.44462
val_loss,8.71441


wandb: Agent Starting Run: st0i0xsh with config:
wandb: 	batch_size: 128
wandb: 	board_embedding_size: 64
wandb: 	board_intermediary_channels: 384
wandb: 	context_length: 256
wandb: 	conv_modules_count: 2
wandb: 	ff_inner_channels: 192
wandb: 	lr: 0.06355237426719156
wandb: 	model_name: model_residual_encoder
wandb: 	num_epochs: 4
wandb: 	num_heads: 8
wandb: 	optimizer: sgd
wandb: 	past_boards: 2
wandb: 	predict_sentences: 10
wandb: 	samples_per_train_epoch: 100000
wandb: 	sp_vocab: 900
wandb: 	stride_big_sequences: 256
wandb: 	text_embedding_size: 64
wandb: 	transformer_blocks: 4
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: Ctrl + C detected. Stopping sweep.
